# PyMC3 implementation of the radon model

Import libraries

In [1]:
import arviz as az
import pymc3 as pm
import numpy as np
import ujson as json

Load data

In [2]:
with open("radon.json", "rb") as f:
    radon_data = json.load(f)

radon_data = {
    key: np.array(value) if isinstance(value, list) else value
    for key, value in radon_data.items()
}

Model definition

In [3]:
coords = {
    "level": ["basement", "floor"],
    "obs_id": np.arange(radon_data["y"].size),
    "county": radon_data["county_name"],
    "g_coef": ["intercept", "slope"],
}

with pm.Model(coords=coords) as radon_model:
    floor_idx = pm.Data("floor_idx", radon_data["x"], dims="obs_id")
    county_idx = pm.Data("county_idx", radon_data["county"], dims="obs_id")
    uranium = pm.Data("uranium", radon_data["u"], dims="county")

    # Hyperpriors:
    g = pm.Normal("g", mu=0.0, sigma=10.0, dims="g_coef")
    sigma_a = pm.Exponential("sigma_a", 1.0)

    # Varying intercepts uranium model:
    a = pm.Deterministic("a", g[0] + g[1] * uranium, dims="county")
    za_county = pm.Normal("za_county", mu=0.0, sigma=1.0, dims="county")
    a_county = pm.Deterministic("a_county", a + za_county * sigma_a, dims="county")
    # Common slope:
    b = pm.Normal("b", mu=0.0, sigma=1.0)

    # Expected value per county:
    theta = a_county[county_idx] + b * floor_idx
    # Model error:
    sigma = pm.Exponential("sigma", 1.0)

    y = pm.Normal("y", theta, sigma=sigma, observed=radon_data["y"], dims="obs_id")

    

Sample model

In [4]:
prior_vars = ["za_county", "g", "a_county", "sigma", "b", "sigma_a", "a", "y"]
with radon_model:
    prior = pm.sample_prior_predictive(500, var_names=prior_vars)
    trace = pm.sample(
        1000, tune=500, target_accept=0.99, random_seed=117
    )
    post_pred = pm.sample_posterior_predictive(trace)
    idata = az.from_pymc3(trace, prior=prior, posterior_predictive=post_pred)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, b, za_county, sigma_a, g]


Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 32 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [5]:
idata

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [7]:
idata.to_netcdf("pymc3.nc")

'pymc3.nc'